In [26]:
import numpy as np
import pandas as pd
from time import time
from sklearn import metrics
from collections import defaultdict
import matplotlib.pyplot as plt
import glob


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics.cluster import contingency_matrix, homogeneity_score, completeness_score, adjusted_rand_score, adjusted_mutual_info_score, v_measure_score
from sklearn.metrics.cluster import contingency_matrix

## Import libraries
import re
import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords' )

import string
from string import punctuation
import matplotlib.pyplot as plt

In [27]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
import random
import pandas as pd
import numpy as np
import re, json, datetime, pytz
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from collections import Counter
import nltk

In [28]:
import pytz
import datetime

pst_tz = pytz.timezone('America/Los_Angeles')

In [29]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?|", " ", tweet).split())

In [30]:
def get_data(text, annotate=0):
    with open(text+'.txt', 'r',encoding="utf8") as f:
        retweet_count, followers, place, title, time, fan = [],[],[],[],[],[]
        sentiment_polarity, sentiment_score = [], []
        pst_tz = pytz.timezone('US/Pacific') 
        for line in f:
            tweet = json.loads(line)
            time_now = datetime.datetime.fromtimestamp(tweet['citation_date'])
            retweet_now = tweet['metrics']['citations']['total']
            if tweet['tweet']['lang'] == 'en':
                title.append(clean_tweet(tweet['title']).lower())
                retweet_count.append(tweet['metrics']['citations']['total']) 
                followers.append(tweet['author']['followers'])
                time.append(datetime.datetime.fromtimestamp(tweet['citation_date'], pst_tz))
                place.append(tweet['tweet']['user']['location'])
                fan.append(annotate)

    d = {'title':title, 'retweet_count':retweet_count, 'followers':followers,
         'time':time,'place':place, 'fan_base':fan}
    df = pd.DataFrame(d)
    return df   

In [31]:
file1 = open('tweets_#gopatriots.txt', 'r')
lines = file1.readlines()

In [32]:
count = 0
for line in lines:
    json_object = json.loads(line)
    if json_object['metrics']['citations']['total'] >= 10:
        count = count + 1
print(count)

122


In [52]:
# df_patriots = get_data('tweets_#gopatriots').append(get_data('tweets_#patriots'))
df_patriots = get_data('tweets_#gopatriots')

In [53]:
df_patriots.head(40)

,title,retweet_count,followers,time,place,fan_base
0,LeGarrette Blount does the Ray Lewis Dance Thr...,6,2895.0,2015-01-14 09:45:41-08:00,"Boston, Massachusetts",0
1,With both my father and husband yelling at the...,3,20896.0,2015-01-15 09:11:15-08:00,new york city.,0
2,The girls soccer team is ranked the No 3 team ...,3,500.0,2015-01-14 17:26:29-08:00,"Plantation, FL",0
3,Patriots Champions RT if you agree GOPATRIOTS,2,491.0,2015-01-16 08:55:32-08:00,Dol Guldur.,0
4,12 0 The girls varsity soccer team dominates t...,6,500.0,2015-01-14 17:26:19-08:00,"Plantation, FL",0
5,thanks for the follow GoPatriots,2,34.0,2015-01-14 05:17:19-08:00,"Boston, Massachusetts",0
6,ria84 which team do you follow who do you want...,4,10955.0,2015-01-14 01:50:11-08:00,"Adelaide, South Australia",0
7,Reminder Old HomeBase Bell Schedule today tomo...,3,1762.0,2015-01-14 02:38:39-08:00,"Powell, Ohio",0
8,Can t wait to watch the Super Bowl of this sea...,1,600.0,2015-01-14 04:50:53-08:00,"Liverpool, ah no, Chitré D:",0
9,DESIGNS Bx Thanks Great day to you too brotha ...,2,1425.0,2015-01-14 06:21:57-08:00,"Kawasaki,Kanagawa,Japan",0


In [41]:
df_hawks = get_data('tweets_#gohawks', 1)

In [42]:
df_hawks.head(40)

,title,retweet_count,followers,time,place,fan_base
0,I lt 3 our defense GoHawks,5,1752.0,2015-01-17 10:19:38-08:00,Ontario,1
1,twelfth dogs are ready gohawks dogslife,2,258.0,2015-01-14 10:18:56-08:00,"Redmond, WA",1
2,Oh no big deal just NFC West Champs and the nu...,5,22.0,2015-01-16 20:21:59-08:00,Seattle,1
3,Good luck at Michigan Jim Harbaugh GoHawks,2,22.0,2015-01-16 20:18:56-08:00,"Buffalo, Wyoming",1
4,GoHawks Keep your eyes on the ball Peyton,2,22.0,2015-01-16 20:16:16-08:00,"McChord AFB, WA",1
5,One of my Favorite Hawk logos that went around...,2,22.0,2015-01-16 20:15:44-08:00,Washington state,1
6,HAPPY NEW YEAR EVERYONE Here s to a 2015 full ...,3,186.0,2015-01-16 00:08:22-08:00,#Seahawks #Mariners,1
7,My hometown GoHawks pics Seattle Washington USA,2,22.0,2015-01-16 20:13:01-08:00,Pacific Northwest - USA,1
8,Absolutely GoHawks,8,22.0,2015-01-16 20:12:16-08:00,The Halls of Valhalla,1
9,12 s are the champions 24 7 TGIBF WhyNotUsAgai...,5,977.0,2015-01-16 18:32:08-08:00,,1


In [54]:
print(len(df_patriots))
print(len(df_hawks))

12884
142367


In [55]:
## Import libraries
import re
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords' )

import string
from string import punctuation

stop_words_skt = text.ENGLISH_STOP_WORDS
stop_words_en = stopwords.words('english')

In [85]:
stop_words_game = ['patriots', 'hawks', 'gopatriots', 'gohawks', 'patriot', 'hawk']

In [87]:
combined_stopwords = set.union(set(stop_words_en),set(punctuation),set(stop_words_skt), set(stop_words_game))
wnl = nltk.wordnet.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

def penn2morphy(penntag):
    """ 
    Converts Penn Treebank tags to WordNet. 
    """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize_sent(list_word): 
    '''
    Returns lemmatized set of tokens with pos tagging
    '''
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]

def stem_rmv_punc(doc): # this should have been at the sentence-level because the pos-tag performs best at sentence-level
    return (word for word in lemmatize_sent(analyzer(doc)) if word not in combined_stopwords and not word.isdigit())

def clean_tokens_lemma (tokens: list):
    '''
    Cleans set of tokens at sentence level by applying lemmatization at sentence level
    '''
    lower_txt = [token.lower() for token in tokens]
    remove_words = [token for token in lower_txt if (not token.isdigit())\
                     and (token not in combined_stopwords) and (len(token)>1)]
    lemmatize_tokens = lemmatize_sent(remove_words) 
    return lemmatize_tokens

def doc_tokens_lemma (doc):
    '''
    Split the document at sentence level, clean it and return clean set of tokens at doc level
    '''
    doc = clean(doc)
    list_sentences = sent_tokenize(doc)
    doc_tokens = []
    for sentence in list_sentences:
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        tokens = nltk.word_tokenize(sentence)
        tokens = clean_tokens_lemma(tokens)
        doc_tokens.extend(tokens)
#     print(doc_tokens)
    return (word for word in doc_tokens)     

def doc_tokens_lemma_woClean (doc):
    '''
    Perform lemmatization without any text cleaning
    '''
    list_sentences = sent_tokenize(doc)
    doc_tokens = []
    for sentence in list_sentences:
        tokens = nltk.word_tokenize(sentence)
        tokens = lemmatize_sent(tokens)
        doc_tokens.extend(tokens)
#     print(doc_tokens)
    return (word for word in doc_tokens)   

def doc_tokens_stem (doc):
    '''
    Clean full text using stemming. This function does not require split at sentence level.
    '''
    doc = clean(doc)
    doc = doc.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(doc)
    lower_txt = [token.lower() for token in tokens]
    remove_words = [token for token in lower_txt if (not token.isdigit())\
                     and (token not in combined_stopwords) and (len(token)>1)]
    stem_tokens = [stemmer.stem(token) for token in remove_words] 
    return (word for word in stem_tokens)

def doc_tokens_stem_woClean (doc):
    '''
    Clean full text using stemming. This function does not require split at sentence level.
    '''
    tokens = nltk.word_tokenize(doc)
    lower_txt = [token.lower() for token in tokens]
    stem_tokens = [stemmer.stem(token) for token in lower_txt] 
    return (word for word in stem_tokens)
    
def doc_tokens (doc):
    '''
    Clean full text without any stemming or lemmatization
    '''
    doc = clean(doc)
    doc = doc.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(doc)
    lower_txt = [token.lower() for token in tokens]
    remove_words = [token for token in lower_txt if (not token.isdigit())\
                     and (token not in combined_stopwords) and (len(token)>1)]
    return (word for word in remove_words)
    
def doc_tokens_woClean (doc):
    '''
    Clean full text using stemming. This function does not require split at sentence level.
    '''
    tokens = nltk.word_tokenize(doc)
    lower_txt = [token.lower() for token in tokens]
    return (word for word in lower_txt)

In [88]:
def clean(text):
    '''
    Helps remove many HTML artefacts from the crawler's output.
    '''
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    if texter == "":
        texter = ""
    return texter

def clean_data(data):
    for index,row in data.iterrows():
        row['full_text'] = clean(row['full_text'])
    return data

In [58]:
from sklearn.model_selection import train_test_split

def count_train_test(train,test):
    
    train_num = train.shape[0]
    test_num = test.shape[0]
    return train_num,test_num

In [59]:
train_patriots, test_patriots = train_test_split(df_patriots, test_size=0.2, random_state = 42)

train_num,test_num = count_train_test(train_patriots,test_patriots)
print("train number:{}".format(train_num))
print("test number:{}".format(test_num))

train number:10307
test number:2577


In [60]:
train_hawks, test_hawks = train_test_split(df_hawks, test_size=0.2, random_state = 42)

train_num,test_num = count_train_test(train_hawks, test_hawks)
print("train number:{}".format(train_num))
print("test number:{}".format(test_num))

train number:113893
test number:28474


In [61]:
## For Patriots

count_vectorizer_patriots = CountVectorizer(min_df=3, stop_words='english', analyzer=doc_tokens_lemma, max_df=0.7)
transformer = TfidfTransformer()

X_train_counts_patriots = count_vectorizer_patriots.fit_transform(train_patriots['title'])
X_train_tfidf_patriots = transformer.fit_transform(X_train_counts_patriots)

X_test_counts_patriots = count_vectorizer_patriots.transform(test_patriots['title'])
X_test_tfidf_patriots = transformer.transform(X_test_counts_patriots)

print('TF-IDF processed train data shape:', X_train_tfidf_patriots.shape)
print('TF-IDF processed test data shape:', X_test_tfidf_patriots.shape)

TF-IDF processed train data shape: (10307, 1915)
TF-IDF processed test data shape: (2577, 1915)


In [51]:
## For Hawks

count_vectorizer_hawks = CountVectorizer(min_df=3, stop_words='english', analyzer=doc_tokens_lemma, max_df=0.7)
transformer = TfidfTransformer()

X_train_counts_hawks = count_vectorizer_hawks.fit_transform(train_hawks['title'])
X_train_tfidf_hawks = transformer.fit_transform(X_train_counts_hawks)

X_test_counts_hawks = count_vectorizer_hawks.transform(test_hawks['title'])
X_test_tfidf_hawks = transformer.transform(X_test_counts_hawks)

print('TF-IDF processed train data shape:', X_train_tfidf_hawks.shape)
print('TF-IDF processed test data shape:', X_test_tfidf_hawks.shape)

TF-IDF processed train data shape: (113893, 10374)
TF-IDF processed test data shape: (28474, 10374)


In [70]:
train = train_patriots.append(train_hawks.sample(n=len(train_patriots)))
test = test_patriots.append(test_hawks.sample(n=len(train_patriots)))

In [71]:
train = train.drop_duplicates()
test = test.drop_duplicates()

In [72]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from tempfile import mkdtemp
from shutil import rmtree
import joblib

In [79]:
cachedir = mkdtemp()
memory = joblib.Memory(location=cachedir, verbose=10)

pipeline = Pipeline([
    ('vect', CountVectorizer(min_df=3, analyzer=doc_tokens_lemma)),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(random_state=42)),
    ('clf', GaussianNB()),
],
memory=memory
)

param_grid = [
    {
        'vect': [CountVectorizer(min_df=3, analyzer=doc_tokens_lemma),
        ],
        'reduce_dim': [
                        TruncatedSVD(n_components=500, random_state=42),
#                         NMF(n_components=500, init='random', random_state=42),
        ],
        'clf': [
                SVC(kernel='linear', C=10, random_state=42),
                LogisticRegression(penalty='l2', C=10, random_state=42),
                GaussianNB(),
                RandomForestClassifier(max_depth=10, random_state=42)
        ],
    }
]

In [80]:
grid = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, scoring='accuracy')
grid.fit(train.title, train.fan_base)
rmtree(cachedir)

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(CountVectorizer(analyzer=<function doc_tokens_lemma at 0x7fbc689839d0>,
                min_df=3), 
7089      The Seahawks better not win this Super Bowl Go...
8473      Yall seahawk fans quiet yall aint making no no...
12112                    patriots GoPatriots Finish the Job
12181                         SuperBowl reaction GoPatriots
8794                      Don t let me down Pats gopatriots
                                ...                        
70978           How many yall going to the send off GoHawks
119004             I hope the Seahawks beat Seattle GoHawks
112548              GoHawks watching the from Venezuela Yay
22201                    That 25 guy is pretty good GoHawks
108966    If you do not let the world know who you re ch...
Name: title, Length: 13742, dtype: object, 
7089      0
8473      0
12112     0
1218

[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmpdlzmtq4_/joblib/sklearn/pipeline/_fit_transform_one/194d490c15417623e9e73a70faa8af1b
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.1s, 0.0min    : Loading _fit_transform_one from /tmp/tmpdlzmtq4_/joblib/sklearn/pipeline/_fit_transform_one/02f2e34dbc40f71c57daf7cae6d37661
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.1s, 0.0min    : Loading _fit_transform_one from /tmp/tmpdlzmtq4_/joblib/sklearn/pipeline/_fit_transform_one/d544a430f388a85d93e221cfa85a8fba
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmpdlzmtq4_/joblib/sklearn/pipeline/_fit_transform_one/0f73db2d45bf1af8d54e515a8a2c49e2
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.1s, 0.0min    : Loading _fit_transform_one from /tmp/tmpdlzmtq

In [81]:
result = pd.DataFrame(grid.cv_results_)
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_reduce_dim,param_vect,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,8.308131,0.082243,2.752050,0.024338,"SVC(C=10, kernel='linear', random_state=42)","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': SVC(C=10, kernel='linear', random_stat...",0.999563,0.999418,0.999418,0.999466,0.000069,1
1,0.211244,0.005096,2.636499,0.009563,"LogisticRegression(C=10, random_state=42)","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': LogisticRegression(C=10, random_state=...",0.999272,0.999127,0.999127,0.999175,0.000069,2
2,0.188560,0.049809,2.665139,0.018704,GaussianNB(),"TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': GaussianNB(), 'reduce_dim': TruncatedS...",0.687864,0.684762,0.667152,0.679926,0.009121,4
3,12.995217,0.083480,2.703887,0.012373,"RandomForestClassifier(max_depth=10, random_st...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': RandomForestClassifier(max_depth=10, r...",0.997526,0.997526,0.997089,0.997380,0.000206,3


In [82]:
best_list = {}
k = 1
for i,row in enumerate(result['rank_test_score']):
    if row >= 1 and row <= 5:
        best_list[row] = i
    
print("The index of 5 best combination:{}".format(best_list))

The index of 5 best combination:{1: 0, 2: 1, 4: 2, 3: 3}


In [83]:
i = 1
while i <= 4:
    index = best_list[i]
    print("_" * 80)
    print("rank_test_score:")
    print(result['rank_test_score'][index]) 
    print("_" * 20)
    print("mean_test_score:")
    print(result['mean_test_score'][index])     
    print("_" * 20)
    print("model parameters:")
    print(result['params'][index])  
    print("_" * 80)
    i += 1

________________________________________________________________________________
rank_test_score:
1
____________________
mean_test_score:
0.9994663773615744
____________________
model parameters:
{'clf': SVC(C=10, kernel='linear', random_state=42), 'reduce_dim': TruncatedSVD(n_components=500, random_state=42), 'vect': CountVectorizer(analyzer=<function doc_tokens_lemma at 0x7fbc689839d0>,
                min_df=3)}
________________________________________________________________________________
________________________________________________________________________________
rank_test_score:
2
____________________
mean_test_score:
0.9991753130349887
____________________
model parameters:
{'clf': LogisticRegression(C=10, random_state=42), 'reduce_dim': TruncatedSVD(n_components=500, random_state=42), 'vect': CountVectorizer(analyzer=<function doc_tokens_lemma at 0x7fbc689839d0>,
                min_df=3)}
________________________________________________________________________________
__

In [89]:
## Try by removing keywords

In [95]:
cachedir = mkdtemp()
memory = joblib.Memory(location=cachedir, verbose=10)

pipeline = Pipeline([
    ('vect', CountVectorizer(min_df=3, analyzer=doc_tokens_lemma)),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(random_state=42)),
    ('clf', GaussianNB()),
],
memory=memory
)

param_grid = [
    {
        'vect': [CountVectorizer(min_df=3, analyzer=doc_tokens_lemma),
        ],
        'reduce_dim': [
                        TruncatedSVD(n_components=500, random_state=42),
                         NMF(n_components=500, init='random', random_state=42),
        ],
        'clf': [
                SVC(kernel='linear', C=10, random_state=42),
                 LogisticRegression(penalty='l2', C=10, random_state=42),
                 GaussianNB(),
                 RandomForestClassifier(max_depth=10, random_state=42)
        ],
    }
]

In [96]:
grid2 = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, scoring='accuracy')
grid2.fit(train.title, train.fan_base)
rmtree(cachedir)

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(CountVectorizer(analyzer=<function doc_tokens_lemma at 0x7fbc68983430>,
                min_df=3), 
7089      The Seahawks better not win this Super Bowl Go...
8473      Yall seahawk fans quiet yall aint making no no...
12112                    patriots GoPatriots Finish the Job
12181                         SuperBowl reaction GoPatriots
8794                      Don t let me down Pats gopatriots
                                ...                        
70978           How many yall going to the send off GoHawks
119004             I hope the Seahawks beat Seattle GoHawks
112548              GoHawks watching the from Venezuela Yay
22201                    That 25 guy is pretty good GoHawks
108966    If you do not let the world know who you re ch...
Name: title, Length: 13742, dtype: object, 
7089      0
8473      0
12112     0
1218

______________________________________________fit_transform_one - 430.9s, 7.2min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/194d490c15417623e9e73a70faa8af1b
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/c6de3af9cca9fc6c1a0fc62812e5bebb
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(NMF(init='random', n_components=500, random_state=42), <13743x2605 sparse matrix of type '<class 'numpy.float64'>'
	with 61920 stored elements in Compressed Sparse Row format>, 
6436      0
1805      0
2288      0
11736     0
7047      0
         ..
70978     1
119004    1
112548    1
22201     1
1

/home/shivam/.local/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


_____________________________________________fit_transform_one - 644.1s, 10.7min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/34158a0e020066669ba47f5a2e66f651
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/dccef523d46fd62e8e61b0bc98cc6dec
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/833d29a9d1f3df179332ede20b01b213
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/194d490c15417623e9e73a70faa8af1b
___________________________________fit_transform_one cache loaded - 0.0s

/home/shivam/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/34158a0e020066669ba47f5a2e66f651
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.1s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/dccef523d46fd62e8e61b0bc98cc6dec
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.1s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/833d29a9d1f3df179332ede20b01b213
___________________________________fit_transform_one cache loaded - 0.1s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv2_/joblib/sklearn/pipeline/_fit_transform_one/194d490c15417623e9e73a70faa8af1b
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.1s, 0.0min    : Loading _fit_transform_one from /tmp/tmp99tunv

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(CountVectorizer(analyzer=<function doc_tokens_lemma at 0x7fbc68983430>,
                min_df=3), 
6436      Lettttzzzz go Game time boom SuperBowl49 GoPat...
1805      Is there a mercy rule in the NFL My god what a...
2288      I don t like either team but for sake of compe...
11736     Seahawks some sore losers bro tryna fight and ...
7047      For the love of God please stop putting a Kard...
                                ...                        
70978           How many yall going to the send off GoHawks
119004             I hope the Seahawks beat Seattle GoHawks
112548              GoHawks watching the from Venezuela Yay
22201                    That 25 guy is pretty good GoHawks
108966    If you do not let the world know who you re ch...
Name: title, Length: 20614, dtype: object, 
6436      0
1805      0
2288      0
1173

/home/shivam/.local/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


_____________________________________________fit_transform_one - 896.6s, 14.9min


/home/shivam/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [118]:
result2 = pd.DataFrame(grid2.cv_results_)
pd.DataFrame(grid2.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_reduce_dim,param_vect,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,37.267668,0.251887,11.704723,0.099588,"SVC(C=10, kernel='linear', random_state=42)","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': SVC(C=10, kernel='linear', random_stat...",0.747963,0.753020,0.738029,0.746337,0.006227,4
1,570.100207,88.016414,77.853300,61.368086,"SVC(C=10, kernel='linear', random_state=42)","NMF(init='random', n_components=500, random_st...",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': SVC(C=10, kernel='linear', random_stat...",0.748545,0.752001,0.743123,0.747890,0.003654,3
2,0.572136,0.119720,2.342068,0.006577,"LogisticRegression(C=10, random_state=42)","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': LogisticRegression(C=10, random_state=...",0.750582,0.753165,0.743123,0.748957,0.004258,2
3,0.661059,0.044667,68.292213,60.461573,"LogisticRegression(C=10, random_state=42)","NMF(init='random', n_components=500, random_st...",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': LogisticRegression(C=10, random_state=...",0.757421,0.750255,0.745306,0.750994,0.004974,1
4,0.316643,0.034054,2.340587,0.007816,GaussianNB(),"TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': GaussianNB(), 'reduce_dim': TruncatedS...",0.635623,0.630039,0.626110,0.630591,0.003903,8
5,0.403428,0.053601,67.056438,60.749883,GaussianNB(),"NMF(init='random', n_components=500, random_st...",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': GaussianNB(), 'reduce_dim': NMF(init='...",0.724243,0.732062,0.727259,0.727855,0.003220,5
6,14.516446,0.075953,2.375185,0.007645,"RandomForestClassifier(max_depth=10, random_st...","TruncatedSVD(n_components=500, random_state=42)",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': RandomForestClassifier(max_depth=10, r...",0.726572,0.733372,0.718527,0.726157,0.006068,6
7,1.566802,0.025442,67.548493,60.777820,"RandomForestClassifier(max_depth=10, random_st...","NMF(init='random', n_components=500, random_st...",CountVectorizer(analyzer=<function doc_tokens_...,"{'clf': RandomForestClassifier(max_depth=10, r...",0.715367,0.710086,0.716490,0.713981,0.002792,7


In [109]:
best_list = {}

In [121]:
best_list_2 = {}
k = 1
for i_iter,row_iter in enumerate(result2['rank_test_score']):
    if row_iter >= 1 and row_iter <= 5:
        best_list[row_iter] = i_iter
    
print("The index of 5 best combination:{}".format(best_list))

The index of 5 best combination:{7: 5, 4: 0, 3: 1, 2: 2, 1: 3, 5: 5}


In [123]:
i = 1
while i <= 5:
    index = best_list[i]
    print("_" * 80)
    print("rank_test_score:")
    print(result2['rank_test_score'][index]) 
    print("_" * 20)
    print("mean_test_score:")
    print(result2['mean_test_score'][index])     
    print("_" * 20)
    print("model parameters:")
    print(result2['params'][index])  
    print("_" * 80)
    i += 1

________________________________________________________________________________
rank_test_score:
1
____________________
mean_test_score:
0.7509941579866949
____________________
model parameters:
{'clf': LogisticRegression(C=10, random_state=42), 'reduce_dim': NMF(init='random', n_components=500, random_state=42), 'vect': CountVectorizer(analyzer=<function doc_tokens_lemma at 0x7fbc68983430>,
                min_df=3)}
________________________________________________________________________________
________________________________________________________________________________
rank_test_score:
2
____________________
mean_test_score:
0.7489569406650086
____________________
model parameters:
{'clf': LogisticRegression(C=10, random_state=42), 'reduce_dim': TruncatedSVD(n_components=500, random_state=42), 'vect': CountVectorizer(analyzer=<function doc_tokens_lemma at 0x7fbc68983430>,
                min_df=3)}
_______________________________________________________________________________

In [105]:
np.save("results",result2)